# 02_04 - Creating tasks for the Agent

In [1]:
## import required libraries
import os
from autogen import ConversableAgent, UserProxyAgent
from dotenv import load_dotenv


load_dotenv(override=True) # load environment variables from .env file

True

In [2]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## Manager Proxy Agent

In [3]:
manager_proxy_agent = UserProxyAgent(
    name="Manager",
    system_message="You are a marketing campaign manager. Your role is to provide campaign topics and evaluate the ideas and scripts generated by the creative team.",
    human_input_mode="NEVER",
    code_execution_config=False,
    llm_config=llm_config,
)

## Idea Generator Agent

In [4]:
idea_generator = ConversableAgent(
    name="Idea Generator",
    system_message='''You are a creative marketing expert. Your role is to:
    1. Generate innovative campaign ideas based on the given topic
    2. Suggest potential storylines and characters for video content
    3. Provide a brief outline for each idea
    Present 3-5 unique ideas for each request.''',
    llm_config=llm_config,
    human_input_mode="NEVER"
)


## Script Writer Agent

In [5]:
script_writer = ConversableAgent(
    name="Script Writer",
    system_message='''You are an expert script writer for marketing videos. Your role is to:
    1. Develop full scripts based on the provided campaign ideas
    2. Include dialogue, scene descriptions, and camera directions
    3. Ensure the script aligns with the campaign objectives and target audience
    Aim for scripts of 2-3 minutes in length.''',
    llm_config=llm_config,
    human_input_mode="NEVER"
)


## Script Reviewer Agent

In [6]:
script_reviewer = ConversableAgent(
    name="Script Reviewer",
    system_message='''You are a critical script reviewer. Your role is to:
    1. Analyze scripts for effectiveness, engagement, and alignment with campaign goals
    2. Provide constructive feedback on dialogue, pacing, and overall structure
    3. Suggest improvements or alternatives where necessary
    4. Ensure the script adheres to brand guidelines and target audience preferences
    Be thorough and specific in your feedback.''',
    llm_config=llm_config,
    human_input_mode="NEVER"
)


## Add tasks for the agents

In [7]:
chats = [
    {
        "sender": manager_proxy_agent,
        "recipient": idea_generator,
        "message": 
            "Hello, I need creative ideas for our new marketing campaign. "
            "Our product is eco-friendly home cleaning products. "
            "Could you generate 3-5 innovative campaign ideas, including potential "
            "storylines and characters for video content?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Summarize the generated ideas "
                             "into a JSON object: "
                             "{'idea1': {'concept': '', 'storyline': '', 'characters': ''}, "
                             "'idea2': {...}, 'idea3': {...}}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": manager_proxy_agent,
        "recipient": script_writer,
        "message": 
            "Great ideas! Let's develop a script for the following concept: "
            "[INSERT SELECTED IDEA HERE]. "
            "Please write a full script including dialogue, scene descriptions, "
            "and camera directions. Aim for a 2-3 minute video.",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Provide a brief summary of the script "
                             "as a JSON object: "
                             "{'title': '', 'main_characters': [], 'key_scenes': [], "
                             "'estimated_duration': ''}",
        },
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": manager_proxy_agent,
        "recipient": script_reviewer,
        "message": "Please review the following script: [INSERT SCRIPT HERE]. "
                   "Analyze it for effectiveness, engagement, and alignment with "
                   "our eco-friendly cleaning products campaign. Provide constructive "
                   "feedback and suggest improvements.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Summarize the review points "
                             "as a JSON object: "
                             "{'strengths': [], 'weaknesses': [], 'suggestions': []}",
        },
    },
    {
        "sender": manager_proxy_agent,
        "recipient": manager_proxy_agent,
        "message": "Based on the script and review, decide if the script is ready "
                   "for production or needs revisions. If revisions are needed, "
                   "specify the key areas to address.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Summarize the decision "
                             "as a JSON object: "
                             "{'decision': 'ready'/'needs_revision', "
                             "'key_revision_areas': []}",
        },
    },
]

## Start researching

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Manager (to Idea Generator):

Hello, I need creative ideas for our new marketing campaign. Our product is eco-friendly home cleaning products. Could you generate 3-5 innovative campaign ideas, including potential storylines and characters for video content?

--------------------------------------------------------------------------------
Idea Generator (to Manager):

Absolutely, I'd be happy to help with that! Here are five innovative campaign ideas for your eco-friendly home cleaning products, complete with storylines and characters for video content:

### 1. **"The Green Guardians"**
#### Outline:
- **Characters:** The Green Guardians consist of a diverse team of animated eco-warriors, each representing a different aspect of the environment (e.g., Water Wave, Earth Terra, Air Breezy, and Fire Solar)

In [9]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

Sure, here's the summary in a JSON object format:

```json
{
  "idea1": {
    "concept": "Clean Homes, Green Planet",
    "storyline": "Weekly vlogs following a diverse family as they tackle household cleaning challenges using eco-friendly products, comparing traditional and green solutions.",
    "characters": "Mom Sarah, Dad Mike, Teen Rachel, Young Child Tommy"
  },
  "idea2": {
    "concept": "Future Cleaners",
    "storyline": "Kids Lisa and James travel to the future to see the positive impact of eco-friendly cleaning habits, then return to the present to implement these practices in their daily chores.",
    "characters": "Present Kids Lisa and James, Teenage Future Lisa and James"
  },
  "idea3": {
    "concept": "Eco-Impact Tracker",
    "storyline": "A digital tool or app where users log their use of eco-friendly products, track their environmental impact, and earn rewards or badges.",
    "characters": "Animated mascots"
  },
  "idea4": {
    "concept": "Eco-Advocate Program

In [10]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.062595, 'gpt-4o-2024-05-13': {'cost': 0.062595, 'prompt_tokens': 4869, 'completion_tokens': 2550, 'total_tokens': 7419}}, 'usage_excluding_cached_inference': {'total_cost': 0}}


{'usage_including_cached_inference': {'total_cost': 0.0418, 'gpt-4o-2024-05-13': {'cost': 0.0418, 'prompt_tokens': 2693, 'completion_tokens': 1889, 'total_tokens': 4582}}, 'usage_excluding_cached_inference': {'total_cost': 0}}


{'usage_including_cached_inference': {'total_cost': 0.041665, 'gpt-4o-2024-05-13': {'cost': 0.041665, 'prompt_tokens': 3083, 'completion_tokens': 1750, 'total_tokens': 4833}}, 'usage_excluding_cached_inference': {'total_cost': 0}}


{'usage_including_cached_inference': {'total_cost': 0.09496, 'gpt-4o-2024-05-13': {'cost': 0.09496, 'prompt_tokens': 10826, 'completion_tokens': 2722, 'total_tokens': 13548}}, 'usage_excluding_cached_inference': {'total_cost': 0}}


